# 1、Test the main_minibatch.py

In [1]:
from __future__ import absolute_import
import time
import numpy as np
import dataset
from model import RNN
# use_gpu(-1) # -1:cpu; 0,1,2,..: gpu


def define_tensor_size(prompt_id):
    tensor_ranges = {
        1: (50, 50),
        2: (50, 50),
        3: (50, 30),
        4: (50, 30),
        5: (50, 30),
        6: (50, 30),
        7: (50, 50),
        8: (50, 100)
    }
    return tensor_ranges[prompt_id]
e = 0.01
lr = 0.1
drop_rate = 0.
prompt_id = 1
vocab_size = 0  # 0 is define to automated infer vocab-size
sent_len, doc_len = define_tensor_size(prompt_id)  # sent_len is batch_size of tensor
hidden_size = [500]
word_embedding_size = 200
# try: gru, lstm
cell = "gru"
# try: sgd, momentum, rmsprop, adagrad, adadelta, adam, nesterov_momentum
optimizer = "nesterov_momentum"
train_path, dev_path, test_path = './data/fold_0/train.tsv', './data/fold_0/dev.tsv', './data/fold_0/test.tsv'
(train_x, train_masks, train_y), (dev_x, dev_masks, dev_y), (test_x, test_masks, test_y), vocab, vocab_size =\
    dataset.get_data((train_path, dev_path, test_path), prompt_id, vocab_size, doc_len, sent_len)

print "#word size = ", vocab_size
print "#train_x shape = ", train_x[0].shape

print "training..."

Using Theano backend.


#word size =  5516
#train_x shape =  (50L, 50L)
training...


# 2、Test the batch_generator

In [2]:
from sklearn.utils import shuffle
def batch_generator(X, masks, y, doc_num=1):
    epoch = 1
    if doc_num==1:
        X_new, masks_new, y_new = shuffle(X, masks, y)
        print "epoch: "+str(epoch)+" begin......"
        for i in xrange(0,len(X_new),doc_num):
            yield X_new[i], masks_new[i], y_new[i]
    else:
        while(True):
            X_new, masks_new, y_new = shuffle(X, masks, y)
            print "epoch: "+str(epoch)+" begin......"
            for i in xrange(0,len(X_new),doc_num):
                yield np.hstack(X_new[i:i+doc_num]), np.hstack(masks_new[i:i+doc_num]), np.hstack(y_new[i:i+doc_num])
            epoch+=1
train_batch = batch_generator(
    train_x, train_masks, train_y, 32
)
x,m,y_=train_batch.next()
print '#batch shape = ', y_

epoch: 1 begin......
#batch shape =  [  8.  10.  10.   9.  10.   9.   9.   8.   8.  10.  10.   8.   6.   8.   7.
  10.  10.  10.  11.   8.  11.   8.   9.   6.   8.   8.   2.   8.   6.  10.
   7.   8.]


# 3、Test the model

In [33]:
print "compiling..."
from mylayers.encoding_layer import SentEncoderLayer
from mylayers.syntax_attention import SyntaxAttentionLayer
from mylayers.layer_utils import init_weights, init_bias
import theano
import numpy as np
import theano.tensor as T
from optimizers import *
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
# Word_Embdding layer
params=[]
rng = RandomStreams(1234)
embeddings = theano.shared(0.2 * np.random.uniform(
    -1.0, 1.0,(vocab_size, 200)).astype(theano.config.floatX),name='WEmb')  # add one for PADDING at the end
params.append(embeddings)
idxs = T.imatrix()
x=x.astype('int')
mask = T.matrix("mask")
X = embeddings[idxs].reshape((idxs.shape[0], idxs.shape[1], 200))
batch=T.iscalar("batch")
sent_encoder_layer = SentEncoderLayer(rng, X, 200, [200],
                                      'gru', 'hh', 0,
                                      1, doc_len*batch, mask)
layer_input = sent_encoder_layer.activation

params+=sent_encoder_layer.params
# sent_X = layer_input[layer_input.shape[0] - 1, :]
sent_X = layer_input[layer_input.shape[0] - 1, :]
doc_sent_X = T.reshape(sent_X,(batch,50, sent_encoder_layer.hidden_size))

sent_mask = T.switch(mask.sum(axis=0)> 0, 1.0, 0.0)
syntax_vector = init_bias(200, 'syntax_vector')

syntax_att = SyntaxAttentionLayer('2', (batch, doc_len, sent_encoder_layer.hidden_size),
                                  doc_sent_X, sent_mask, syntax_vector)
W_a = init_weights([2*200, 200], "W_a" + '2')
W_u = init_weights([200, 1], "W_u"+'2')
b = init_bias(200, 'b_a'+'2')
#############################################
#    syntax attention
# syntax_matrix = T.reshape(T.tile(syntax_vector, batch*50), (batch,50,200))
# concat = T.concatenate([doc_sent_X, syntax_matrix], axis=2)
# strength = T.dot(T.tanh(T.dot(concat, W_a)+b), W_u).flatten()
# strength_mask = strength*sent_mask
# a = T.nnet.softmax(strength_mask).reshape((batch,50))[:,:,None]
# c = (a*doc_sent_X).sum(axis=1)
##################################################
#   mse
c = syntax_att.activation
W_out = init_weights((200,1), 'mse_W')
b_out = init_bias(1, 'mse_b')
params.append(W_out)
params.append(b_out)
y_pred = T.dot(c, W_out)+b_out

y_true = T.vector(('y_true'))
res = T.pow(y_pred-y_true.reshape((batch,1)), 2).mean()

import theano
clip = theano.gradient.grad_clip(res,0.0,1.0)
gparams = []

for param in params:
    gparam = T.grad(clip, param)
    gparams.append(gparam)
optimizer = eval('sgd')
updates = optimizer(params, gparams)

f = theano.function([idxs, mask, y_true,batch], 
                    res,
                    on_unused_input='ignore',
                    updates=updates,
                    allow_input_downcast=True
                    )
for i in range(6):
    print f(x,np.asarray(m, dtype=theano.config.floatX), y_, 32)
    
# f = theano.function([idxs, mask, y_true], [c,y_pred,res,layer_input[1]],updates=updates, on_unused_input='ignore')
# f = theano.function([idxs, mask, y_true], [c,y_pred,res,layer_input[1]],updates=updates, on_unused_input='ignore')
# 
# for i in range(10):
#     c_, y_pred_ ,r,layer_input_= f(x,np.asarray(m, dtype=theano.config.floatX), y_)
#     # print c_.shape, y_pred_,r,sent_x
#     print layer_input_.shape

compiling...


74.4926834106


48.9988288879


32.6952171326


22.2682476044


15.5997047424


11.334985733


# 4、Test the main_minibatch.py

In [ ]:
import theano
model = RNN(vocab_size, word_embedding_size, hidden_size, cell, 'rmsprop', 0, doc_len)

# model.train(x,np.asarray(m, dtype=theano.config.floatX), 0.1, y_, 32)
from dataset import dev_test_batch_generator
train_batch = dev_test_batch_generator(
    train_x, train_masks, train_y, 32)
# (x_, mask, y_)
count=0
for (x_, mask, y_) in train_batch:
    if count >20:
        break
    # model.predict(x_, np.asarray(
        # mask, dtype=theano.config.floatX), y_, 32)
    print y_.dtype
# for i in range(20):
#     ex,m,y_=train_batch.next()
#     epoch, true_cost, pred=model.train(x,np.asarray(
#         m, dtype=theano.config.floatX),
#         lr, y_, 1)
#     print true_cost

# 5、Test creating vocab

In [6]:
from dataset import create_vocab
vocab = create_vocab('./data/fold_0/train.tsv',8,True,True)

In [7]:
len(vocab)

9987

In [11]:
import theano.tensor as T
import theano
import numpy as np

from mylayers.layer_utils import init_weights, init_bias
b = np.array([[0, 2], [0, 4]]).astype("float32")
a = np.array([1]).astype('float32')
m= T.matrix('hehe')
x= T.tile(m,(1,3))
y=T.switch(x.sum(axis=0) > 0, 1.0, 0.0)
z=theano.shared(np.array([0,0,0,0,0,0],dtype='float32'))
y=T.mean(T.square(z-y))
# y=y[None,:]
# y[...]=b
out = theano.function([m],y)
out(b)

array(0.5, dtype=float32)

In [23]:
import numpy as np
x=np.random.randn((10)) * 0.1
x=np.tile(x,(1,2))
x=x.reshape((2,10))

print x
print x.shape

[[-0.00509187 -0.06839405 -0.00483718  0.09116028  0.02922797  0.08932855
   0.16974536  0.26184026  0.16582535  0.30151564]
 [-0.00509187 -0.06839405 -0.00483718  0.09116028  0.02922797  0.08932855
   0.16974536  0.26184026  0.16582535  0.30151564]]
(2L, 10L)


In [39]:
import nltk
import pprint
import numpy as np
from keras.preprocessing import sequence
def get_sents(string, vocab, doc_len=100, sent_len=50):
    """
    :param string:
    :return: ndarray:shape=(doc_len, sent_len), mask: shape=(doc_len, sent_len), num_hit, unk_hit, total
    """

    num_hit = 0
    unk_hit = 0
    total = 0
    sents = nltk.sent_tokenize(string)
    doc_matrix = np.zeros(shape=(doc_len, sent_len))
    for i, sent in enumerate(sents):
        tokens = nltk.word_tokenize(sent)
        indices = []
        if i>= doc_len:
            break
        for word in tokens:
            indices.append(1)
            total += 1    
        doc_matrix[i,:] = sequence.pad_sequences([indices], sent_len, padding='post')
    return doc_matrix, num_hit, unk_hit, total
x="@ORGANIZATION1, @NUM1 out of @NUM2 people surveyed in @LOCATION1, @CAPS1 agree that the use of computers benefits society. I personally concer" \
             "n with these @NUM1 people, computers have a positive effect on people."
# sents = nltk.sent_tokenize(x)
print get_sents(x,{'out':1, 'people':2})[0].shape

(100L, 50L)


[[2, 3, 4], [1, 2, 3]]
